<h1>Scraping Toronto Postal Codes from Wikipedia</h1>

First we will import a few beneficial libraries.
<br>
* Numpy - array manipulation
* Pandas - DataFrame manipulation
* Requests - easier way to get html source code from a url
* BeautifulSoup - A html parser that will allow us to easily access the data inbetween html tags



In [21]:
import numpy as np
import pandas as pd
import requests as rq
from bs4 import BeautifulSoup as bs

We will store the link to our target page into a variable, url

In [22]:
#Storing URL & page title for potential later use
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
title='List of postal codes of Canada: M'

Using requests.get(targetpage).text we save the html source code of the page into variable html

In [23]:
#scraping plain text html of target page and storing in variable
html=rq.get(url).text

Now we utilize the BeautifulSoup methods with the lxml parsing engine. We use lxml as the page is not very complicated and want to optimize the speed of the calculation.<br><br>
Following that we use the .find method to find the tag that signifies the start of the HTML table (table) with the class wikitable sortable. As there is only one table the new variable will contain the data we are looking for.

In [24]:
#Assign text to soup object then Using beautiful soup package to isolate the wiki table
obj=bs(html,'lxml')
table=obj.find('table',{'class':'wikitable sortable'})

The variable table contains the data we want but not in a format we can use it. HTML builds tables one row at a time using the tr tag, then seperating the columns with the td tag. We can therefore use BeautifulSoup to extract the rows into a single list with the find_all method. After that we can simply run a for loop to create a nested list with each row. Once we have a nested list we can create a pandas dataframe using pd.DataFrame()

In [25]:
#Isolate table rows by html syntax tr
trrows=table.find_all('tr')

In [26]:
#create list object to assign all table rows to
l=[]

#run for loop to pull out each individual row to a seperate list and nest into list l
for tr in trrows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    l.append(row)
    
#Create Dataframe from nested list l
df=pd.DataFrame(l, columns=["Postal Code", "Borough",'Neighborhood'])
df.head()

,Postal Code,Borough,Neighborhood
0,None,None,None
1,M1A\n,Not assigned\n,Not assigned\n
2,M2A\n,Not assigned\n,Not assigned\n
3,M3A\n,North York\n,Parkwoods\n
4,M4A\n,North York\n,Victoria Village\n


At this stage we just need to clean our data set. First of we have a '\n' string attached to every data point. We remove this by using the series.str.replace() method from pandas. This is necessary as in the basestate the columns are all in object format.

In [27]:
#Clean dataset by stripping out \n
df['Postal Code']=df['Postal Code'].str.replace('\n','')
df['Borough']=df['Borough'].str.replace('\n','')
df['Neighborhood']=df['Neighborhood'].str.replace('\n','')

The first row contains no useful data and will be dropped by slicing the first index (0) off the dataframe

In [28]:
#Drop first row with None
df=df[1:]

Optional step here to see how many postal codes are assigned to each borough. it also shows how many rows we will drop containing 'Not assigned'

In [29]:
#see how many Not assigned boroughs exist
group=df.groupby(['Borough']).count()
group

,Postal Code,Neighborhood
Borough,,
Central Toronto,9,9
Downtown Toronto,19,19
East Toronto,5,5
East York,5,5
Etobicoke,12,12
Mississauga,1,1
North York,24,24
Not assigned,77,77
Scarborough,17,17


Using slicing of column borough not equal to not assigned we get a dataframe consisting of only postal codes + neighborhoods with named boroughs.

In [30]:
#Create a final dataframe to store all postal codes, boroughs and neighborhoods where Borough is not 'Not assigned'
df=df[df.Borough != 'Not assigned']
#Reset index sake good order
df=df.reset_index(drop=True)
df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [31]:
#using .shape method to return total number of rows, columns to confirm correctly scraped table
df.shape

(103, 3)

<h1>Part 2 - Longitude + Latitude of Neigborhoods</h1>

Now we want to get the latitude and longitude of each postal code to be able to pass it to foursquare's API.<br><br>
First we import geocoder

In [32]:
!pip install --user geocoder

In [33]:
#from geopy.geocoders import Nominatim
#from geopy.extra.rate_limiter import RateLimiter
import geocoder
import time

#ALTERNATIVE METHOD PART 1
#Initialize Nominatim agent and create a locater with 1 second delay between queries to avoid timing out
#gloc=Nominatim(user_agent='CourseraCapstone')
#locator=RateLimiter(gloc.geocode,min_delay_seconds=2)

Now we need to use our free Bing developer API key to query the Bing service. I have it in a hidden cell to protect it. You can setup a Bing developer basic account at https://www.bingmapsportal.com/

In [42]:
# The code was removed by Watson Studio for sharing.

We initialize two Numpy arrays with 103 zeros (the number of total postal codes). Then run a for loop generating a json file for each query. We use the enumerate function to count the index numbers of the postal code to assign the correct latitude + longitude to the queried postal code.

In [35]:
lati=np.zeros(103)
longi=np.zeros(103)

for i,post in enumerate(df['Postal Code']):
    g=geocoder.bing('{}, Toronto, Ontario'.format(post),key=bingcred)
    lati[i]=g.lat
    longi[i]=g.lng

Finally we create 2 new columns based on our completed series and view the completed Dataframe. End of part 2

In [40]:
df['Latitude']=lati
df['Longitude']=longi
df.head(10)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.751881,-79.330360
1,M4A,North York,Victoria Village,43.730419,-79.312820
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.655140,-79.362648
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.723209,-79.451408
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.664490,-79.393021
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.662769,-79.528313
6,M1B,Scarborough,"Malvern, Rouge",43.811531,-79.195518
7,M3B,North York,Don Mills,43.749290,-79.361687
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.707939,-79.311600
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657360,-79.378181


In [37]:
#ALTERNATIVE METHOD PART 2
#df['query']=df['Postal Code']+', Toronto, Ontario'
#df['location']=df['query'].apply(locator)
#df.drop(['query'],axis=1,inplace=True)
#df.head()